<p> 
<hr>
<i><H5>SET UP</H5></i>
For this project we aim to use the EC2 Instance Connect End Point to connect to resources in the VPC (Virtual Private Cloud) using the AWS CLI. Once set up an IAM User will be created with programmatic access which will be configured to our computer. This will allow us to use the EC2 Instance connect endpoint to connect to resources in the private subnet and the public subnet of our VPC using AWS CLI
</p>
<br>

<H5><i>Pre-Installation & Requirements</i><H5>
Prerequisites for the project include:

<ul>
<li>Homebrew</li>
<li>iterm2</li>
<li>AWS CLI</li>
</ul>


<hr>

<ol>
<li><i>AWS - IAMS</i></li>
(Creating new Administrative Access User)
<br>
<img src="Images/UsersSelect.png" alt="Drawing" style="width: 300px;"/>

![CreateUser](Images/CreateUser.png)

Once created Attach required policies: 

![UserName](Images/UserName.png)
![AttachPolicies](Images/AttachPolicies.png)


<hr>
<li><i>Secret Access Key for IAM User</i></li>
<br>

![ClickUser](Images/ClickUser.png)
![CreateAccessKey](Images/CreateAccessKey.png)
![CLIAttribute](Images/CLIAttribute.png)
![AccessKeyShow](Images/AccessKeyShow.png)


<hr>
<li><i>Configuring the secret access Key on the AWS CLI and ECE Instance
<br>
This will be needed to utilise AWS resource in the VP
<br>
Open Terminal and enter 
<br>
<code>
> aws configure
</code>

![aws configure](Images/awsConfigure.png)
<br>
Once set up credentials can be found through the following: 
<br>
<code>
> cd
> ls -a 
</code>

<br>
Note: "-a" syntax is used to reveal hidden folders
<br>
<code>
> cd .aws
</code>

<br>
present 2 repositories "config" and "credentials", utilise <i>cat</i> command to access contents
<br>

<code>
> cat config
<br>[default]
<br>region = eu-west
</code>

<br>
Configuration has created a default where the region has been stored

<hr>
<li> Testing AWS CLI Set Up 
Enter the following:
<br> 
<code>
> aws iam get-user
</code>

![Getuser](Images/GetUser.png)
</ol>





Creating a Custom VPC 
We will be creating a 3 tier VPC containing the following 
- Tier 1: 
    Public Subnet holding resources such as
    - NAT Gateway 
    - Load Balancer 
    - Bastion Host 

- Tier 2
    Private Subnet 
    - Web servers
- Tier 3
    Another Private Subnet
    - DataBase 

We will be duplicating the subnet accross multiple availability zones for high availability and Fault Tolerance 
We will also create an Internet Gateway and Custom Route table to allow resources in the VPC to have access to the internet 
We will cover the Route Table's purpose in the VPC 

To create your VPC, you must first need to select the region where you want to create the VPC 

![Search VPC](Images/VPCSearch.png)

Ensure you have the region where you want to create your VPC

![Select Region](Images/VPCRegionSelect.png)


![Create VPC](Images/CreateVPC.png)
Name your VPC 

Enter a IPv4 CIDR of your choice 
![VPC IPv4](Images/VPCName.png)

(What is an IPv4 CIDR)
Classless Inter-Domain Routing (CIDR) notation is a way to represent an IP address network range, including IPv4 addresses. CIDR notation uses a networking routing prefix and a prefix length to represent an IP address
![IPv4Explained](Images/IPv4Explained.png)


![VPC Finalise](Images/VPCIPv4.png)

Once the VPC is created click on Actions 
![Click Actions](Images/EnableDNSHostName.png)

Click the tickbox for Enabling DNS Host Names and Save the feature to your VPC
![Save DNS HostName](Images/SaveEnable.png)

The following step is to set up a Internet Gateway 
This allows us to facilitate the connection between the resources within the VPC and the internet  
Please note that you can only attach one Internat Gateway per VPC

When a VPC is created a default Internet Gateway is created which can be viewed here: 
![Default Internet Gateway](Images/DefaultInternetGateway.png)

When you filter for the VPC just created we can see there are no existing Internet Gateways linked to the Dev VPC created
![Filter for VPC](Images/FilterVPC.png)
![Filtered Gateways](Images/FilteredGateways.png)

Note you can only attach 1 internet Gateway for 1 VPC

You can create an Internet Gateway by clicking the following: 
![Create Internet Gateway](Images/CreateInternetGateway.png)

Once created you can attach a VPC to the selected Internet Gateway by selecting the VPC alias recently created
![Attach Internet Gateway](Images/AttachIGtoVPC.png)
![Select VPC](Images/SelectVPC.png)
![Attached Status](Images/AttachedStatus.png)






The next step is creating a subnet for the VPC. 
This will allow us to logically place AWS resources based on different needs (Private Subnets and Public Subnets)

Private Subnets we will use to add: 
- EC2 Instances 
- RDS Instance

Public Subnet:
- ALB
- NAT Gateway

We are going to start by creating a Subnet:
![](Images/CreateSubnet.png)
![](Images/Select%20DEV%20VPC.png)
![](Images/EnterSubnetDetails.png)

Using the same steps as above we will create 2 more subnets so the following applies: 
(In the same AWS Region as the VPC: eu-west-2a)
1. Public Subnet - IPv4 CIDR: 10.0.0.0/24
2. Private App Subnet - IPv4 CIDR: 10.0.2.0/24
3. Private Data Subnet - IPv4 CIDR: 10.0.4.0/24

We are going to have a mirror image of these subnets in a second availability zone (eu-west-2b) with the following properties: 
1. Public Subnet - IPv4 CIDR: 10.0.1.0/24
2. Private App Subnet - IPv4 CIDR: 10.0.3.0/24
3. Private Data Subnet - IPv4 CIDR: 10.0.5.0/24

Ensure all subnets are located in the same availability zone

Next we will enable auto assign on the Public subnets
![](Images/EnableAutoAssign.png)

This ensures AWS automatically assigns a Public IPv4 to any resources launched in these subnets, this allows AWS to automatically assign a public IPv4 address to any resource launced in the subnet. 


Creating a Route Table 

To make a subnet public: 
1. Create a route table (containing a route to the internet)
2. Subnets then associated with the route table will become public

![](Images/CreateRouteTable.png)

Note: When a VPC is created AWS automatically creates a main route table (as the default route table in that VPC)

In the creation of a route table you can select a VPC which will be used with the Route table 
![](Images/SelectVPCForRouteTable.png)

Our next step is to add a route to the route table

Route are specified under the routes section of the selected Route Table. 
In the below example we have a Target set to "local" which means it will only be routing traffic within the VPC. 
![](Images/DefaultRoute.png)

In order to create a route to the Internet we must first edit route 
![](Images/EditRoute.png)

Note: In networking any time we want traffic to have access to the internet the destination becomes 0.0.0.0/0 
Under target click Internet Gateway and select the gateway associated with the Dev Internet Gateway
![](Images/RouteToInternet.png)

Now that a route has been addedd to the route table to route traffic to the internet, any subnet associated with the specific route table will become public. So in order to associate the Public subnets with the route table : 
- Select Subnet associations 

![](Images/EditSubnetAssociation.png)

- Edit Subnet associations

![](Images/SelectPublicSubnets.png)



Main Takeways: 
To make a subnet public you have to create a route table containg a route that is routing traffic to the internet (0.0.0.0/0) once created you associate the route table with the subnets. 

The remaining Subnets (The private subnets will be private by default as they already have route table associated with the private subnets this route table is the main route table)

Any time you create subnets in your VPC and don't explicitely establish the subnets with a route table AWS will associate those subnets with the main route table making them private 

AWS Automatically creates a main route table in your VPC when you create a subnet in your VPC and it's private by default. 
This can be confirmed because in the main route table you can view that the traffic is not routed to the Internet it has a local target

![](Images/LocalRoutes.png)

You can also view the subnet associations from the main route table we can see the Private Subnets have been associated as the subnets were not explicitely assocaiate with them

![](Images/PrivateSubnetsAssociations.png)

<hr>

NAT GATEWAY 

Nat Gateway allows resources in the private subnet of a VPC to have access to the internet. 
When we put any instances (i.e. EC2 Instance or RDS Instance) in a private subnet, resources will not have access to the internet by default. 

There's cases where an EC2 Instance may need access to the internet to download packages on to it.
Note: if we want an EC2 Instance to have access to the internet (i.e. to download packages) but don't want users from the internet to have access to the EC2 Instance we use a NAT Gateway 

![](Images/NatGateway.png)

In a production Environment it best practice to create a NAT Gateway in each availability zone to ensure high availability and fault tolerance.

In this example we will create 1 NAT Gateway to associate all the private subnets with the NAT Gateway.

To create a NAT Gateway 
1. Filter for VPC you are working with 

![](Images/FilterNATGateways.png)

2. Click Create NAT Gateway 

![](Images/CreateNATGateway.png)

3. NAT Gateway Setting

Note: ensure the subnet you create NAT Gateway in is a public subnet 
and we are going to allocate an Elastic IP for the NAT Gateway

![](Images/NATGatewaySettings.png)

<hr>

ROUTE TABLE 

When creating a private subnets in a VPC any resources created in the private subnets will not have access to the Internet, hence a route table is required to route traffic to the internet through the NAT Gateway.

To create the Route table ensure the state of your Nat Gateway is available 

![](Images/AvailableStateNatGateway.png)

Select route tables
Filter by VPC and you should see only 2 route tables : 
- The main route table (created by default by AWS)
- The Public Route table we created

Create Route Table 

![](Images/CreateRouteTable.png)

Update the setting to ensure route table is set up in the Dev VPC 

![](Images/PrivateRouteTableSettings.png)

On the Routes tab you should see that the route table is only routing traffic locally in the VPC (hence private)

![](Images/EditRoute.png)

We're going to create a route with the internet as Destination (0.0.0.0/0) and the NAT Gateway as a target 

![](Images/EditRouteNATGateway.png)

The next step is to associate the created route table with the private subnets.

1. Click Subnet Associations and Edit Subnet Associations
![](Images/SubnetAssociation.png)

2. Select all the private subnets and save associations
![](Images/PrivateSubnetAssociationswithRouteTable.png)

The difference between a private subnet and a public subnet. 

A private subnet will have a route table, routing traffic to the internet THROUGH THE NAT GATEWAY 
<br>
A public subnet will have a route table, routing traffic to the internet THROUGH THE INTERNET GATEWAY

<hr>

SECURITY GROUPS 

Security groups are the firewalls we will use to regulate traffic within our VPC. 

1. Application Load Balancer Security Group <br>
    - Traffic will come from anywhere on the Internet on Port 80 and Port 443 <br>

<br>

2. EC2 Instance Connect Endpoint Security Group<br>
    Attached to the EC2 Instance Connect End Point  <br>
    - Only OUTBOUND Traffic on Port 22 (the source of the traffic is only going to come from the CIDR block of the VPC )

<br>

3. App Server Security Group <br>
    Attached to the Web servers <br>
    - Traffic from Application Load Balancer Security group on Port 80 and Port 443
    - Traffic from EC2 Instance Security Group on Port 22

<br>

4. Database Security Group <br>
    Attached to the RDS Instance <br>
    - Traffic from App Server Security Group on Port 3306 

<br>

5. Elastic File System Security Group<br>
    Attached to the Elastic File System
    - Traffic from App Server Security Group and Elastic File System Security Group on Port 2049
    - Traffic from EC2 Instance Connect Endpoint Security Group on Port 22
    

To create the security group for our website we will enter the VPC dashboard 

Filter for our DEV VPC 

![](Images/FilterVPCClickSecurity.png)

Once filtered click on Security groups and Create Security Groups

![](Images/CreateSecurityGroup.png)

1. Application Load Balancer Security Group 
- Under VPC Info we will select the Dev VPC 
- Create 2 Inbound Rules for HTTP(Port 80) and HTTPS(Port 443) for the source being anywhere on the internet (0.0.0.0/0)

![](Images/SecurityGroupInbound.png)

- Ensure outbound rules are on All traffic and the destination is anywhere on the internet (0.0.0.0/0)

![](Images/ALBSecurityOutbound.png)

2. EC2 Instance Connect Endpoint Security Group 

![](Images/EICESecurityGroup.png)

The EICE Security group we will only have an outbound rule (NO INBOUND RULES) 
- Select SSH Type (Port 22) with the destination limited to the CIDR block of our VPC (this can be retrieved in the VPC Console)

![](Images/VPCCIDRblock.png)

![](Images/EICEOutbound.png)


3. App Server Security Group
For the App Server Security Group we are going to limit traffic to and from the Application Load Balance Security Group

![](Images/AppServerSecurityGroup.png)

- For Inbound Traffic Select HTTP(Port 80) and HTTPS(Port 443) with the source limited to the application Loag Balancer Security Group

![](Images/AppServerSecurityInboundRule.png)

- Add Inbound SSH (Port 22) from the EC2 Instance Connect Endpoint Security group 

![](Images/AppServerSecurityGroupSSH.png)

- For Outbound we will allow all types of traffic with the destination being anywhere on the internet (0.0.0.0/0)

![](Images/AppServerSecurityGroupOutbound.png)

4. Database Security Group
For Database Security we will allow traffic to the MySQL Port 

![](Images/DatabaseSecurityGroup.png)

- Add an Inbound Security Group rule of Type MYSQL/Auror (Port 3306) with the Source being the App Server Security Group 

![](Images/DatabaseSecurityGroupInbound.png)

- For Outbound Rules allow all traffic with the destination being anywhere on the Internet (0.0.0.0/0)

![](Images/AppServerSecurityGroupOutbound.png)

5. Elastic File System Security Group 

![](Images/EFSSecurityGroup.png)

- Set up Inbound rule of type NFS (Port 2049) with the source as App Server Security Group 

![](Images/EFSSecurityGroupInboundRule1.png)

- Set up Inbound rule of type SSH (Port 22) with the source as the EC2 Instance Connect Endpoint Security Group

![](Images/EFSSecurityGroupInboundRule2.png)

- For Outbound rules set up all traffic types with destination anywhere on the internet

![](Images/AppServerSecurityGroupOutbound.png)

Once the EFS Security group has been created we need to allow inbound traffic of type NFS from the source as the EFS Security group. 

This can be done by editing the inbound rules

![](Images/EFSSecurityGroupEditInboundRules.png)

![](Images/EFSSecurityGroupInboundRule3.png)


Note:The rule to allow inbound traffic from the EFS Security Group to itself as multiple instances will be directly accessing the EFS at the same time so the EFS must allow traffic to itself in order to allow multiple EC2 Instances to gain access from it

<hr>
EC2 INSTANCE CONNECT ENDPOINT

The EC2 Instance Connect Endpoint allows us to connect to any resource in the public or private subnets in the VPC without haveing to manage SSH Keys

To create an EC2 Instance Connect Endpoint in the VPC: 
- Select Endpoint in the VPC Dashboard 

![](Images/SelectEndpoints.png)

- Create Endpoint

![](Images/CreateEndpoint.png)

- Select EC2 Instance Connect Endpoint Service Category 

![](Images/EndpointSettings.png)

- Select the Dev VPC and tick the Security group creted for the EICE

![](Images/EndpointSecurityGroupSetup.png)

- We want to create the EC2 Instance Connect Endpoint in a private Subnet 

![](Images/EndpointSubnetSelection.png)

(Note it may take a short while for the Status of the Endpoint to become available )

Once the Status becomes available we are going to launch an EC2 Instance in a private subnet and connect it to the EICE 

To Launch EC2 Instance
Locate the EC2 Dashboard by searching EC2 on the Management Console 

On the Left Panel menu select "Instances" and click "Launch Instances" 

![](Images/InstancesSelect.png)
![](Images/LaunchInstances.png)

- Ensure you have the Amazon Linux Instance selected in the Instance settings

![](Images/EC2AmazonLinux.png)

- Ensure the instance type is a t2.micro instance type

![](Images/EC2t2InstanceType.png)

- A Key Pair will not be required as we are using the EICE to SSH into the instance 

![](Images/EC2ProceedWithoutPair.png)

- In the Network settings of the EC2 Instance, amend the subnet to ensure a Private subnet is selected
(We will leave the auto-assign public IP option as disabled) this won't be needed as we're launching the EC2 Instance in the private subnet

![](Images/EC2NetworkSettings.png)

- Firewall settings will be set up so that the EC2 Instance will have the App Server Security Group 

![](Images/EC2InstanceSecurityGroup.png)

This should provide us with the following Summary

![](Images/EC2Summary.png)

Once you launc the instance click on the instance and wait till the state is Running and Status check have been passed to proceed. 

![](Images/EC2InstanceState&Status.png)

To test we are able to connect the the EC2 Instance we launched, In the Instances section tick the box for the instance and click connect 

![](Images/EC2Connect.png)

On the EC2 Instance Connect Setting Page ensure the connection type is under "Connect using EC2 Instance Connect Endpoint" and the DEV EICE is associated

![](Images/EICESelect.png)

![](Images/ConnectToInstanceSettings.png)

You should get the following terminal open as following: 

![](Images/EC2Terminal.png)

We have successfully connected to the EC2 Instance in the private subnet using the EC2 Instance Connect Endpoint, through this terminal we can run commands to install packages on the EC2 Instance

<code>
> sudo yum update -y 
</code>

This is used to update packages in the EC2 Instances, this command can be used to test the EC2 instance has internet access to download required packages 

If the output is as follows: 

<code>
> sudo yum update -y <br>
Last metadata expiration check: 0:19:02 ago on Fri Oct  4 14:47:09 2024. <br>
Dependencies resolved.<br>
Nothing to do.<br>
Complete!<br>
</code>

There are no required packages to install

We can also use the AWS CLI to connect to the EC2 Instance that was launched.
Open Terminal 

First check you have aws cli isntalled in the computer by entering the following: 

<code>
> aws --version
</code>

If you get the following response: 

<code>
> aws --version
aws-cli/2.17.27 Python/3.11.9 Darwin/22.6.0 source/x86_64
</code>

Enter the following:

(Collect the Instance ID from the launched EC2 on the EC2 Dashboard)

![](Images/FilterSpecificEC2.png)

![](Images/copyEC2ID.png)

<code>
>aws ec2-instance-connect ssh --instance-id i-0ee0000a0a0a0000aa
</code>

It may ask you if you are sure you want to continue connecting, enter 'yes' and you should receive the following output : 

<code>
>aws ec2-instance-connect ssh --instance-id i-0ee0000a0a0a0000aa
<pre>
   ,     #_ 
   ~\_  ####_        Amazon Linux 2023
  ~~  \_#####\
  ~~     \###|
  ~~       \#/ ___   https://aws.amazon.com/linux/amazon-linux-2023
   ~~       V~' '->
    ~~~         /
      ~~._.   _/
         _/ _/
       _/m/'
Last login: Fri Oct  4 15:02:15 2024 from 10.0.3.115
</pre>
</code>

We are going to install apache on our EC2 Instance enter: 

<code>
>sudo yum install httpd -y
</code>

You should get the outcome to end with something similar to  
<code>
<pre>
Installed:
  apr-1.7.2-2.amzn2023.0.2.x86_64                                               
  apr-util-1.6.3-1.amzn2023.0.1.x86_64                                          
  apr-util-openssl-1.6.3-1.amzn2023.0.1.x86_64                                  
  generic-logos-httpd-18.0.0-12.amzn2023.0.3.noarch                             
  httpd-2.4.62-1.amzn2023.x86_64                                                
  httpd-core-2.4.62-1.amzn2023.x86_64                                           
  httpd-filesystem-2.4.62-1.amzn2023.noarch                                     
  httpd-tools-2.4.62-1.amzn2023.x86_64                                          
  libbrotli-1.0.9-4.amzn2023.0.2.x86_64                                         
  mailcap-2.1.49-3.amzn2023.0.3.noarch                                          
  mod_http2-2.0.27-1.amzn2023.0.3.x86_64                                        
  mod_lua-2.4.62-1.amzn2023.x86_64 
</pre>
</code>

By now we have confirmed the following: 
- We can SSH into any EC2 Instance in our VPC Using the EC2 Instance Conned Endpoint
- We can also use the Management Console and the AWS CLI to connect to the EC2 Instance via the EC2 Instance Connect Endpoint 

To terminate the EC2 Instance:
- Select the EC2 Instance 

![](Images/FilterSpecificEC2.png)

- Select Instance State and Terminate (delete) instance and click Terminate on the Pop-up

![](Images/TerminateEC2Instance.png)

The End result should be as such:
 
![](Images/EC2Terminated.png)




<hr>

ELASTIC FILE SYSTEM

The EFS allows multiple Web Servers to access the application code from the same storage

Elastick Block Storage can only be attached to one EC2 Instance, i.e. if we launch 2 web servers using Elastic Block Store we would have to install the website on each instance separately, changes made on 1 EC2 Instance would not be reflected on the other EC2 Instance 

![](Images/EBSDiagram.png)

The EFS will allow both web servers to access the application code from the same storage, the wordPress code in the EFS so both EC2 Instances can access the code at the same time, any changes made to the code will reflect on both instances

![](Images/EFSDiagram.png)

<hr>

To create an Elastic File System search "EFS" in the Management Console and "create file system"

![](Images/CreateEFS.png)

On the pop-up click "customise" 

Create a "Regional" File System Type with enabled automatic backups and enabled encryption of data at rest

![](Images/EFSSettings.png)

![](Images/EFSLifecycleManagement.png)

Leaving the Performance settings as default:

![](Images/EFSPerformanceSettings.png)

In the network setting we will specify the Dev VPC and the mount targets 

Mount targets are NFSv4 endpoint in which you can mount an EFS, in this section we specify the subnets we want to put our mount targets. 

In this case ensure the mount targets are in the private Data subnets and the EFS Security groups have been selected for the mount targets

![](Images/EFSNetworkAccess.png)

The File system policy can be left as default. 

Once the setting have been reviewed you can create the EFS, you should end up with a page similar to : 

![](Images/EFSCreated.png)

You can click on the File SystemID to select the EFS, 

![](Images/DevEFS.png)

You can see the DNS name of the file system, you can also mount the EFS to a web directory by clicking the attach 

![](Images/EFSAttach.png)

This will provide the command to mount the EFS to any web servers

![](Images/EFSAttachToWeb.png)


<hr>

DATABASE CREATION

To create the Database in the Private Database Subnet

Search "RDS" in the Management Console Search box

Before creating the Database we must first create a subnet group (allows us to specify which subnets we want to have our database in)

![](Images/SubnetGroupSelect.png)

![](Images/CreateSubnetGroup.png)

Ensure in the Subnet group details you specify the Dev VPC 

![](Images/SubnetGroupDetails.png)

Remember our Private Data Subnets are located in (eu-west-2a : Private Data Subnet - IPv4 CIDR: 10.0.4.0/24 , eu-west-2b : Private Data Subnet - IPv4 CIDR: 10.0.5.0/24 ) hence the CIDR selected 

Once selected create the subnets

The following step is to create the RDS Instance, go to "Databases" on the left panel and click "create database"

![](Images/SelectDatabaseMenu.png)

![](Images/CreateDatabase.png)

Create a database with the following: 
- "Standard Create" method , "MySQL" engine 

![](Images/DatabaseCreationMethod.png)

- Select the latest version of MySQL  (at the time of this creation it's 8.0.39), Dev/Test Template (intended for development use outside of a production environment)

![](Images/DatabaseEngineVersionTemplates.png)

- To create a Standby Database we would select Multi-AZ DB Instance (but for the sake of avoiding additional charges select Single DB Instance)

![](Images/DatabaseAvailability&Durability.png)


- Set up Login credentials

![](Images/DBLoginCredentials.png)

- In instance configuration include previous generation classes

![](Images/InstanceConfiguration.png)

- For Storage we will have Provisioned IOPS SSD (io1) as the Storage Type 

![](Images/DBStorageSettings.png)

- For connectivity ensure the Dev VPC is selected

![](Images/DBConnectivitySettings.png)

- Select existing VPC security group associated with the Database Security Group, we're going to set up the single DB Instance in eu-west-2b

![](Images/DBConnectivityFirewall.png)

- Leave Password Authentication and Monitoring as default

![](Images/DBPasswordAuth&Monitoring.png)

- Set up additional configuration create a name for your database and leave remaining encryption settings as default

![](Images/AdditionalConfiguration.png)

Create the Database. 

Once available you will be able to select the DB identifier and retrieve the database endpoint 

![](Images/DBEndpoint.png)

<hr>

APPLICATION LOAD BALANCER 

This will allow the users to access the website created on our server. 
The servers will be placed in the private subnet (so users from the internet won't be able to access it) 
Servers will be added to a target group and the Application Load Balancer will route traffic to the target group 


First we need to create a target group (which will contain the private servers)

In the EC2 Dashboard within the Load balancing section select "Target Groups" and "Create Target Group"

![](Images/SelectTargetGroups.png)

![](Images/CreateTargetGroup.png)

Website will be hosted on an EC2 Instance hence we leave the target group type as "Instances" name the target group

![](Images/TGConfiguration.png)

Select the Development VPC 

![](Images/TGVPCSelect.png)

Leave the health checks as default and Next

![](Images/TGDefaultHealthChecks.png)

In the registered targets we would register the servers to the target groups (we will amend this once the servers have been launched)

![](Images/TGRegisteredTargets.png)

Click "Create Target Group"


With the Target Groups created we can launch the EC2 Instance in the private subnet so that we may add the EC2 Instance to the Target Group. 

Select Instances in the EC2 Dashboard and Launch Instances 

![](Images/InstancesSelect.png)

![](Images/LaunchInstances.png)

- Ensure you have the Amazon Linux Instance selected in the Instance settings

![](Images/EC2AmazonLinux.png)

- Ensure the instance type is a t2.micro instance type

![](Images/EC2t2InstanceType.png)

- A Key Pair will not be required as we are using the EICE to SSH into the instance 

![](Images/EC2ProceedWithoutPair.png)

- In the Network settings of the EC2 Instance, amend the subnet to ensure a Private subnet is selected
(We will leave the auto-assign public IP option as disabled) this won't be needed as we're launching the EC2 Instance in the private subnet

![](Images/EC2NetworkSettings.png)

- Firewall settings will be set up so that the EC2 Instance will have the App Server Security Group 

![](Images/EC2InstanceSecurityGroup.png)

Leave storage as default and launch the Instance 

Wait for the instance to pass the status check before you proceed. 

To add the web server to the Target group 

Select "Target Groups" in the Load Balancing section of the EC2 Dashboard

![](Images/SelectTargetGroups.png)

Select the Target group we created earlier (DEV-TG)

![](Images/DEV-TG.png)

In the Targets tab of your Development Target group you can register Targets 

![](Images/RegisterTargets.png)

Check the Available Instances for the Instance relating to the Web Server 

![](Images/IncludeAvailableInstances.png)

The EC2 Instance will appear in the Review Targets box, once you have verified the EC2 Instance has been addedd to the Review Targets

Register pending targets

![](Images/RegisterPendingTargets.png)


The next step is creating the Application Load Balancer used to route traffic to the servers in the target group. 

In the Load Balancing Section of the EC2 Dashboard select "Load Balancers" and "Create Load Balancer"

![](Images/SelectLoadBalancers.png)

![](Images/CreateLoadBalancer.png)

We are going to create an Application Load Balancer

![](Images/CreateApplicationLoadBalancer.png)

For Basic Configuration we the ALB will be internet-facing with address type IPv4

![](Images/ALBConfiguration.png)

Map the ALB to the Development VPC 

Note: The application Load Balancer will only reach the public Subnets so select Public Subnets in both Availability Zones

![](Images/ALBNetworkMapping.png)

Under Security groups we will remove the default security group and select the ALB Security Group created earlier 

![](Images/ALBSecurityGroupSelect.png)

In the Listeners and routing section, 

(The HTTP Listener is going to be on Port 80)

This listener will route traffic to our Target group

![](Images/ALBListener.png)

NOTE:Once we register the SSL Certificate a HTTPS Listener can be created to redirect traffic from HTTP to HTTPS

![](Images/ALBSummary.png)

Review the summary, Ensure the following: 
* Internet-facing on IPv4
* Appliation Load Balancer Security Group selected 
* Network mapped tot he public Subnets 
* HTTP listener on Port 80 and routing traffic to the target group 

Once reviewed create the Application Load Balancer

<hr>

To install the WordPress on EC2 

<a href= "https://docs.aws.amazon.com/linux/al2/ug/hosting-wordpress.html">Host a WordPrss blog on AL2</a>

This documentaion will work you through the steps required to host a WordPress Website on the EC2 Instace providing specific requirements 

you can also search "wordpress requirements" to find out we need:
* PHP version 7.4+
* MySQL version 5.7+
* HTTPS (Apache/Nginx)

Use the "install-wordpress.sh" for the commands required to install wordrpess on the EC2 Instace. 


<table>
  <tr>
    <th>Command</th>
    <th>Definition</th>
  </tr>
  <tr>
    <td><code>sudo su</code></td> 
    <td>Change the user's permission to root user, which will grant the privilege to install packages on the EC2 Instance</td>
  </tr>
  <tr>
    <td><code>sudo yum update -y</code></td>
    <td>Update all the packages on the EC2 Instance</td>
  </tr>
  <tr>
    <td><code>sudo mkdir -p /var/www/html</code></td>
    <td>Create an HTML Directory</td>
  </tr>
  <tr>
    <td><code>EFS_DNS_NAME=fs-03f0bf26349956311.efs.eu-west-2.amazonaws.com</code></td>
    <td>Update the value of the specify the DNS name of the EFS</td>
  </tr>
  <tr>
    <td><code><pre>sudo mount -t nfs4 -o nfsvers=4.1,
rsize=1048576,wsize=1048576,
hard,timeo=600,retrans=2,
noresvport "$EFS_DNS_NAME":/ /var/www/html</pre>
    </code></td>
    <td>Command required to mount the EFS to the HTML Directory</td>
  </tr>
  <tr>
    <td><code>sudo yum install -y httpd</code></td><td>Install Apache Server on the EC2 Instance</td>
  </tr>
  <tr><td><code>sudo systemctl enable httpd</code></td><td>Will enable the instance to start on boot (When EC2 is restarted Apache server will be booted) 
  </td></tr>
  <tr><td><code>sudo systemctl start httpd</code></td><td>Install the Apache Server on the EC2 Instance</td></tr>
  <tr>
    <td><code><pre>sudo dnf install -y \
php \
php-cli \
php-cgi \
php-curl \
php-mbstring \
php-gd \
php-mysqlnd \
php-gettext \
php-json \
php-xml \
php-fpm \
php-intl \
php-zip \
php-bcmath \
php-ctype \
php-fileinfo \
php-openssl \
php-pdo \
php-tokenizer</pre></code></td>
<td>Installing php and all the necessary extensions </td>
  </tr>
  <tr>
    <td><code>sudo wget https://dev.mysql.com/get/mysql80-community-release-el9-1.noarch.rpm </code></td>
    <td>Install MySQL v.8 on EC2 Instance</td>
  </tr>
  <tr>
    <td><code><pre>sudo dnf install -y mysql80-community-release-el9-1.noarch.rpm 
sudo rpm --import https://repo.mysql.com/RPM-GPG-KEY-mysql-2023
sudo dnf repolist enabled | grep "mysql.*-community.*"
sudo dnf install -y mysql-community-server </pre></code></td>
<td>Installs MySQL on EC2 Instance</td>
  </tr>
  <tr>
    <td><code><pre>sudo systemctl start mysqld
sudo systemctl enable mysqld</pre></code></td>
<td>Installs MySQL on EC2 Instance</td>
  </tr>
  <tr>
    <td><code><pre>sudo usermod -a -G apache ec2-user
sudo chown -R ec2-user:apache /var/www
sudo chmod 2775 /var/www && find /var/www -type d -exec sudo chmod 2775 {} \;
sudo find /var/www -type f -exec sudo chmod 0664 {} \;
chown apache:apache -R /var/www/html </pre></code></td>
<td>Sets permissions on the EC2 Instance (retrieved from the AWS Documention)</td>
  </tr>
  <tr>
    <td><code><pre>wget https://wordpress.org/latest.tar.gz
tar -xzf latest.tar.gz
sudo cp -r wordpress/* /var/www/html/</pre></code></td>
<td>download the WordPress zip file, unzip and copy the WordPress Application code from the wordPress directory into the html directory
last line ensures we are copying all the application code on to the EFS </td>
  </tr>
  <tr>
    <td><code>sudo cp /var/www/html/wp-config-sample.php /var/www/html/wp-config.php</code></td>
    <td>Connect WordPress Website to the RDS Instance, creates a copy of the WordPress Configuration file 
and will save copy under "wp-config.php" and will be updated with the RDS credentials to connect WordPress to the RDS Instance</td>
  </tr>
  <tr>
    <td><code>sudo vi /var/www/html/wp-config.php</code></td>
    <td>Using VI to modify the configuration file "wp-cofig.php" and update the wp-config.php with the credentials of the RDS Database  </td>
  </tr>
  <tr>
    <td><code>sudo service httpd restart</code></td>
    <td>Restart the Apache folder</td>
  </tr>
</table>

These command will need to be run on the EC2 Instance to install the WordPress Application 




